# Visualization at Scale
---

When working with large datasets, it's crucial to consider the performance of each visualization method. This notebook investigates the performance and data fidelity of the previously covered methods using high-resolution grids.

## Benchmarked Dataset

The dataset used for timings and examples in this notebook were provided from the [DYAMOND Initiative](https://www.google.com/search?q=dyamond&oq=dyamond&gs_lcrp=EgZjaHJvbWUyCQgAEEUYORiABDIGCAEQIxgnMgcIAhAuGIAEMgkIAxAuGAoYgAQyCQgEEAAYChiABDIHCAUQLhiABDIHCAYQLhiABDIJCAcQABgKGIAEMgcICBAAGIAEMg8ICRAAGAoYgwEYsQMYgATSAQc5NjRqMGo0qAIAsAIA&sourceid=chrome&ie=UTF-8). 

There are four datasets, each of the same grid, but with different resolutions. The table below summarizes the scale of these datasets.

| Element / Resolution | 30km      | 15km      | 7.5km      | 3.75km     |
|----------------------|-----------|-----------|------------|------------|
| Faces                | 655,362   | 2,621,442 | 10,485,762 | 41,943,042 |
| Nodes                | 1,310,720 | 5,242,880 | 20,971,520 | 83,886,080 |




## Timing Comparison

Timings were taken on a single [NCAR Derecho Node](https://arc.ucar.edu/knowledge_base/74317833). All results are in seconds. 

### Data Processing (Initial Run)

| Visualization Method / Grid Resolution  | 30km        | 15km        | 7.5km      | 3.75km      |
|-----------------------------------------|-------------|-------------|------------|-------------|
| Polygon Raster (Including Antimeridian) | 28.5 | 122.2       | 463        | 1990        |
| Polygon Raster (Excluding Antimeridian) | 1.69 (0.23) | 5.96 (0.09) | 23.1 (0.52) | 93 (1.01)   |
| Point Raster                            | 0.13 (0.03) | 0.16 (0.01) | 0.35 (0.00) | 1.08 (0.07) |

### Data Processing (Subsequent Runs)

| Visualization Method / Grid Resolution  | 30km        | 15km        | 7.5km       | 3.75km       |
|-----------------------------------------|-------------|-------------|-------------|--------------|
| Polygon Raster (Including Antimeridian) | 0           | 0           | 0           | 0            |
| Polygon Raster (Excluding Antimeridian) | 0.30 (0.00) | 1.02 (0.36) | 3.46 (0.01) | 13.60 (0.08) |
| Point Raster                            | 0.13 (0.03) | 0.16 (0.01) | 0.35 (0.00) | 1.08 (0.07)  |

### Visualization 

| Visualization Method / Grid Resolution | 30km | 15km | 7.5km | 3.75km |
|----------------------------------------|------|------|-------|--------|
| Polygon Raster (0.5 Pixel Ratio)       | 0    | 0    | 0     | 0      |
| Polygon Raster (1.0 Pixel Ratio)       | 0    | 0    | 0     | 0      |
| Polygon Raster (4.0 Pixel Ratio)       | 0    | 0    | 0     | 0      |
| Point Raster (0.5 Pixel Ratio)         | 0    | 0    | 0     | 0      |
| Point Raster (1.0 Pixel Ratio)         | 0    | 0    | 0     | 0      |
| Point Raster (4.0 Pixel Ratio)         | 0    | 0    | 0     | 0      |

## Polygon vs Point Rasters

### Global

```Python
uxds['relhum_200hPa'][0].plot.rasterize(method='polygon', 
                                        width=1000, height=500, exclude_antimeridian=True, 
                                        clim=clim, 
                                        title="Global Polygon Raster")
```

![Polygon Raster](../images/plots/global_polygon_raster.png)

```Python
uxds['relhum_200hPa'][0].plot.rasterize(method='point', 
                                        width=1000, 
                                        height=500, 
                                        clim=clim, 
                                        title="Global Point Raster")
```


![Point Raster](../images/plots/global_point_raster.png)


### Regional

```Python
uxds['relhum_200hPa'][0].plot.rasterize(method='polygon', 
                                        width=1000, 
                                        height=500, 
                                        exclude_antimeridian=True, 
                                        dynamic=False, 
                                        xlim=(-68, -60), 
                                        ylim=(-71, -66), 
                                        clim=clim, 
                                        title="Regional Polygon Raster")
```

![Polygon Raster](../images/plots/regional_polygon_raster.png)

```Python
uxds['relhum_200hPa'][0].plot.rasterize(method='point', 
                                        width=1000, 
                                        height=500, 
                                        dynamic=False, 
                                        xlim=(-68, -60), 
                                        ylim=(-71, -66), 
                                        clim=clim, 
                                        title="Regional Point Raster")
```

![Point Raster](../images/plots/regional_point_raster.png)

Without specific and additional parameters, both the Polygon and Point raters look identical. 

However, once we set `dynamic=True`, which dynamically re-rasterizes as we zoom and pan across our plot, we can begin to see the differences in these methods

```Python
uxds['relhum_200hPa'][0].plot.rasterize(method='polygon', width=1000, height=500, exclude_antimeridian=True, clim=clim, title="Global Polygon Raster")
```

```Python
uxds['relhum_200hPa'][0].plot.rasterize(method='polygon', 
                                        width=1000, 
                                        height=500, 
                                        exclude_antimeridian=True, 
                                        dynamic=True, xlim=(-68, -60), 
                                        ylim=(-71, -66), 
                                        clim=clim, 
                                        title="Regional Polygon Raster (Dynamic)")
```

![Polygon Raster](../images/plots/regional_polygon_raster_dyn.png)


```Python
uxds['relhum_200hPa'][0].plot.rasterize(method='point', 
                                        width=1000, 
                                        height=500, 
                                        dynamic=True, xlim=(-68, -60), 
                                        ylim=(-71, -66), 
                                        clim=clim, 
                                        title="Regional Point Raster (Dynamic)")
```

![Point Raster](../images/plots/regional_point_raster_dyn.png)

The Polygon Raster can be zoomed in indefinitely, since our polygon mesh completely covers the screen. However, since our we are using shaded coordinate points for our Point Raster, once we zoom in, we no longer have completely filled cells, we are left with the individual points.

## Pixel Ratio Comparison